# Кластеры похожих Web-сайтов на основе word2vec модели

В этом файле показан пример работы с кластерами сайтов, созданных с помощью word2vec.

Все домены разбиты на 10000 кластеров и 1000 укрупненных кластеров (суперкластеров).

Кластеры получены на основе оценки сходства сайтов по использованию их посетителями в одном контексте (несколько упрощенно: сходство указывает на наличие общей аудитории сайтов).

Исходная word2vec модель содержит 198156 доменов, наиболее посещаемых по данным LiveInternet.

In [1]:
import pandas as pd
import numpy as np
import os
import IPython

# загрузка кластерной модели и определение функций для работы с ней:

def init_domains_clucters():
    def log(s):        
        IPython.display.clear_output(wait = True)
        print(s)
        
    if not os.path.exists("domain_clusters_dcrp065cos.csv"):
        log("загрузка из HDFS файла domain_clusters_dcrp065cos.csv")
        os.system("hadoop fs -get /user/amelnikov/models/domain_clusters_dcrp065cos.csv domain_clusters_dcrp065cos.csv")
    
    log("чтение файла domain_clusters_dcrp065cos.csv")
    dcl = pd.read_csv("domain_clusters_dcrp065cos.csv", header = 0, sep = ';', index_col = False)

    # Возвращает DataFrame с доменами, находящихся в одном кластере с заданным доменом domain
    def get_cluster(domain): 
        row = dcl[dcl.domain == domain]
        if len(row) == 0:
            return pd.DataFrame({'error': ['Домен "%s" не найден' % domain]})
        return dcl.loc[dcl.cluster == row.cluster.iloc[0], :].loc[:,['domain']]

    # Возвращает DataFrame с доменами, находящихся в одном укрупненном кластере (суперкластере) с заданным доменом domain
    def get_supercluster(domain): 
        row = dcl[dcl.domain == domain]
        if len(row) == 0:
            return pd.DataFrame({'error': ['Домен "%s" не найден' % domain]})
        return dcl.loc[dcl.supercluster == row.supercluster.iloc[0], :].loc[:,['domain']]
    
    log("")    
    return (get_cluster, get_supercluster)

(get_cluster, get_supercluster) = init_domains_clucters()

## Примеры использования

Домены сайтов следует указывать без www, http и других фрагментов URL !!!
Если пытаемся найти домены похожий на тот, что отсутствует в модели, выдается таблица с ошибкой:

In [2]:
get_cluster('несуществующий-домен.рф')

,error
0,"Домен ""несуществующий-домен.рф"" не найден"


Задаем домен сайта, для которого нужно получить похожие домены:

In [3]:
url_domain = 'kanobu.ru'

In [4]:
# получаем таблицу доменов, находящихся в одном кластере с заданным доменом:
d_cl = get_cluster(url_domain)

#смотрим таблицу:
d_cl 

#если нужно посмотреть весь список доменов:
#d_cl.domain.tolist()

,domain
660,stopgame.ru
780,forums.playground.ru
986,gamebomb.ru
1023,kanobu.ru
1145,igromania.ru
1204,coop-land.ru
1486,gamer.ru
1505,evrl.to
1789,gmbox.ru
1975,shazoo.ru


In [5]:
# получаем таблицу доменов, находящихся в одном укрупненном кластере с заданным:
d_scl = get_supercluster(url_domain)

#смотрим таблицу:
d_scl

#если нужно посмотреть весь список доменов:
#d_scl.domain.tolist()

,domain
660,stopgame.ru
780,forums.playground.ru
986,gamebomb.ru
1023,kanobu.ru
1145,igromania.ru
1204,coop-land.ru
1486,gamer.ru
1505,evrl.to
1789,gmbox.ru
1975,shazoo.ru


## Как сохранить результаты в файл

In [6]:
# чтобы не возникала ошибка UnicodeDecodeError при сохранении результатов в файл:
import sys 
reload(sys)  
sys.setdefaultencoding('utf8')

In [7]:
# сохраняем в файл MS Excel все записи:
d_scl.to_excel("supercluster.xlsx", index = False)

In [8]:
# сохраняем в CSV файл все записи (кодировка UTF-8, если необходима другая - задать параметр encoding):
d_scl.to_csv("supercluster.csv", index = False)